In [1]:
import numpy as np
import sys
import itertools as itertools
import time
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Heisenberg import *
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy
from python.Backend import Backend

In [2]:
n_sites = 50
XY_coupling = 1.0

In [3]:
device = "numpy"
# device = "torch"

bk = Backend(device)

NKeep = 50
NSweep = 20
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

In [4]:
import numpy.typing as npt
import pickle
from dataclasses import dataclass, field, asdict, replace
from copy import deepcopy
from typing import Any, Optional
from pathlib import Path


@dataclass
class Params:
    L: int
    XY: float
    ZZ: float
    h: float
    NKeep: int
    NSweep: int
    diag: bool
    Krylov: int
    cutoff: float
    two: bool
    
    def to_log(self) -> str:
        log_dict = asdict(self)
        
        log_dict['XY'] = round(self.XY, 2)
        log_dict['ZZ'] = round(self.ZZ, 2)
        log_dict['h'] = round(self.h, 2)
        log_dict['cutoff'] = round(self.cutoff, 3)
        
        # Filter out keys where the value is None
        filtered_log_dict = {key: log for key, log in log_dict.items() if log is not None}
        
        # Create log string
        return "_".join(
            f"{key}={log}" for key, log in filtered_log_dict.items()
        )

@dataclass
class Results:
    ground_MPS: list[npt.NDArray]
    ground_state_energy: float
    Neumann_entropy: float
    ground_energies: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    spin_z: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    spin_correlations: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    
    
    def to_array(self):
        return replace(
            self, **{field: np.array(getattr(self, field))
            if isinstance(getattr(self, field), list) else getattr(self, field)
            for field in asdict(self)}
        )
        
    def self_destruct(self):
        for field_name in self.__dataclass_fields__:
            field_value = getattr(self, field_name)
            if isinstance(field_value, list):
                field_value.clear()  # Clear list contents
            else:
                setattr(self, field_name, None)  # Set to None if not a list



In [5]:
ZZ_coupling_start = -5
ZZ_coupling_end = 5
delta_ZZ = 0.1

ZZ_couplings = bk.arange(ZZ_coupling_start, ZZ_coupling_end+delta_ZZ, delta_ZZ)

magnetic_field_start = -10
magnetic_field_end = -5
delta_h = 0.1

magnetic_fields = bk.arange(magnetic_field_start, magnetic_field_end+delta_h, delta_h)

In [6]:
for it1, ZZ_coupling in enumerate(ZZ_couplings):
    for it2, magnetic_field in enumerate(magnetic_fields):
        
        if verbose:
            print(f"ZZ={round(ZZ_coupling,2)} | h={round(magnetic_field,2)}")
        
        Hamiltonian = XXZ_model(
            bk=bk, n_sites=n_sites, ZZ_coupling=ZZ_coupling,
            XY_coupling=XY_coupling, magnetic_field=magnetic_field,
        )
        
        ground_energies, ground_times, ground_MPS = DMRG(
            bk = bk,
            Hamiltonian = Hamiltonian,
            NKeep = NKeep,
            NSweep = NSweep,
            Krylov_bases = Krylov_bases,
            Lanczos_cutoff = Lanczos_cutoff,
            iterative_diag = True,
            two_site = two_site,
            verbose = verbose,
            tol = 1e-8,
        )
        
        param = Params(
            L = n_sites,
            XY = XY_coupling,
            ZZ = ZZ_coupling,
            h = magnetic_field,
            NKeep = NKeep,
            NSweep = NSweep,
            diag = iterative_diag,
            Krylov = Krylov_bases,
            cutoff = Lanczos_cutoff,
            two = two_site
        )
        
        # dir_name = Path(f"data")
        dir_name = Path(f"{device}/data")
        dir_name.mkdir(parents=True, exist_ok=True)
        
        filename = f"{param.to_log()}"
        ground_state_energy = ground_energies[-1]
        Neumann_entropy = get_Neumann_entropy(ground_MPS, bk=bk)
        spin_z = get_spin_z(ground_MPS, bk)
        
        if verbose:
            print(f"computing spin correlations: ", end="")
            now = time.perf_counter()
        spin_correlations = get_spin_correlations(ground_MPS, bk)
        if verbose:
            print(f"{round(time.perf_counter()-now,3)}s")
        
        result = Results(
            ground_MPS = [],
            ground_state_energy = ground_state_energy,
            ground_energies = ground_energies,
            Neumann_entropy = Neumann_entropy,
            spin_z = spin_z,
            spin_correlations = spin_correlations,
        )
        
        output: dict[str, Any] = {}
        output.update(asdict(param))
        output.update(asdict(result))
        
        with open(dir_name / f"{filename}.pkl", "wb") as file:
            pickle.dump(output, file)
        

ZZ=-5.0 | h=-10.0
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-311.25 | time=0js
iter=1 | energy=(-311.24999999999875+0j) | time=0.17854542203713208s
computing spin correlations: 2.149s
ZZ=-5.0 | h=-9.9
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-308.75 | time=0js
iter=1 | energy=(-308.74999999999864+0j) | time=0.20096199202816933s
computing spin correlations: 2.147s
ZZ=-5.0 | h=-9.8
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-306.25 | time=0js
iter=1 | energy=(-306.2499999999986+0j) | time=0.16575199295766652s
computing spin correlations: 2.195s
ZZ=-5.0 | h=-9.7
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-303.75 | time=0js
iter=1 | energy=(-303.74999999999

In [7]:
ZZ_coupling_start = 2.7
ZZ_coupling_end = 5
delta_ZZ = 0.1

ZZ_couplings = bk.arange(ZZ_coupling_start, ZZ_coupling_end+delta_ZZ, delta_ZZ)

magnetic_field_start = -5
magnetic_field_end = 0
delta_h = 0.1

magnetic_fields = bk.arange(magnetic_field_start, magnetic_field_end+delta_h, delta_h)

In [8]:
for it1, ZZ_coupling in enumerate(ZZ_couplings):
    for it2, magnetic_field in enumerate(magnetic_fields):
        
        if verbose:
            print(f"ZZ={round(ZZ_coupling,2)} | h={round(magnetic_field,2)}")
        
        Hamiltonian = XXZ_model(
            bk=bk, n_sites=n_sites, ZZ_coupling=ZZ_coupling,
            XY_coupling=XY_coupling, magnetic_field=magnetic_field,
        )
        
        ground_energies, ground_times, ground_MPS = DMRG(
            bk = bk,
            Hamiltonian = Hamiltonian,
            NKeep = NKeep,
            NSweep = NSweep,
            Krylov_bases = Krylov_bases,
            Lanczos_cutoff = Lanczos_cutoff,
            iterative_diag = True,
            two_site = two_site,
            verbose = verbose,
            tol = 1e-8,
        )
        
        param = Params(
            L = n_sites,
            XY = XY_coupling,
            ZZ = ZZ_coupling,
            h = magnetic_field,
            NKeep = NKeep,
            NSweep = NSweep,
            diag = iterative_diag,
            Krylov = Krylov_bases,
            cutoff = Lanczos_cutoff,
            two = two_site
        )
        
        # dir_name = Path(f"data")
        dir_name = Path(f"{device}/data")
        dir_name.mkdir(parents=True, exist_ok=True)
        
        filename = f"{param.to_log()}"
        ground_state_energy = ground_energies[-1]
        Neumann_entropy = get_Neumann_entropy(ground_MPS, bk=bk)
        spin_z = get_spin_z(ground_MPS, bk)
        
        if verbose:
            print(f"computing spin correlations: ", end="")
            now = time.perf_counter()
        spin_correlations = get_spin_correlations(ground_MPS, bk)
        if verbose:
            print(f"{round(time.perf_counter()-now,3)}s")
        
        result = Results(
            ground_MPS = [],
            ground_state_energy = ground_state_energy,
            ground_energies = ground_energies,
            Neumann_entropy = Neumann_entropy,
            spin_z = spin_z,
            spin_correlations = spin_correlations,
        )
        
        output: dict[str, Any] = {}
        output.update(asdict(param))
        output.update(asdict(result))
        
        with open(dir_name / f"{filename}.pkl", "wb") as file:
            pickle.dump(output, file)
        

ZZ=2.7 | h=-5.0
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-91.925 | time=0js
iter=1 | energy=(-91.9249999999998+0j) | time=0.12159419897943735s
computing spin correlations: 2.077s
ZZ=2.7 | h=-4.9
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-89.425 | time=0js
iter=1 | energy=(-89.4249999999996+0j) | time=0.11380240001017228s
computing spin correlations: 2.099s
ZZ=2.7 | h=-4.8
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-86.925 | time=0js
iter=1 | energy=(-86.92499999999964+0j) | time=0.11385010002413765s
computing spin correlations: 2.115s
ZZ=2.7 | h=-4.7
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-84.425 | time=0js
iter=1 | energy=(-84.42499999999941+0j) | 